In [41]:
# Import Libraries
import os
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import csv
import time
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation 

In [8]:
# Authentication Keys
# https://developer.twitter.com/en/portal/projects/1268606526274580480/apps/18064440/keys

# Think of these as the user name and password that represents
# your Twitter developer app when making API requests. 
consumerKey = '9gjPeGkMZj2935QAVEGEFqZG2'
consumerSecret = 'jSJbncxF8OifGepXcoR5uadfvhlVxXgoO9P5zDtMzNQidimRYx'

# User-specific credentials used to authenticate OAuth 1.0a API requests. 
# They specify the Twitter account the request is made on behalf of.
accessToken = '1130514658673213441-4GKVmzpYX67W2HlNiccXVIR3e0kzi7'
accessTokenSecret = 'HBN0T2QDg2nOcFOlZP0GyS9MwYGkjb3gtsrE7ebxKnJn8'

In [9]:
# Create the authentication object
auth = tweepy.OAuthHandler(consumerKey, consumerSecret) 
    
# Set the access token and access token secret
auth.set_access_token(accessToken, accessTokenSecret) 

# Creating the API object while passing in auth information
api = tweepy.API(auth)

In [33]:
# Constants 
START_DATE = '2020-05-01'
TWEET_NUMBER = 100
SEARCH_WORD = '#CRA'
RATE_LIMIT = 180
SLEEP_TIME = 900/180 # 15 minutes = 900 seconds

In [37]:
# Collect tweets using Cursor method
# http://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html
def buildTestSet():
    tweet_list = []
    tweets_fetched = tweepy.Cursor(api.search,
                  q=SEARCH_WORD,
                  lang='en',
                  since=START_DATE).items(5) 

    for tweet in tweets_fetched:
        tweet_list.append(tweet.text)
        # print(tweet.text)
        
    # Array where Test Set is stored
    return tweet_list

In [38]:
# Build the test set
testDataSet = buildTestSet()

In [51]:
def buidTrainingSet(corpusFile, tweetDataFile):
    corpus = []    
    with open(corpusFile,'rb') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',', quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2], "label":row[1], "topic":row[0]})
            
    trainingDataSet = []

    for tweet in corpus:
        try:
            status = api.GetStatus(tweet["tweet_id"])
            print("Tweet fetched" + status.text)
            tweet["text"] = status.text
            trainingDataSet.append(tweet)
            time.sleep(SLEEP_TIME) 
        except: 
            continue
    # now we write them to the empty CSV file
    with open(tweetDataFile,'wb') as csvfile:
        linewriter = csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet["label"], tweet["topic"]])
            except Exception as e:
                print(e)
    return trainingDataSet

In [57]:
"""
corpusFile = "./corpus.csv"
tweetDataFile = "./tweetDataFile.csv"
"""

trainingData = buidTrainingSet(corpusFile, tweetDataFile)

FileNotFoundError: [Errno 2] No such file or directory: './corpus.csv'

In [56]:
class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]
    
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(trainingData)
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

NameError: name 'trainingData' is not defined

In [13]:
def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [14]:
def extract_features(tweet):
    tweet_words=set(tweet)
    features={}
    for word in word_features:
        features['contains(%s)' % word]=(word in tweet_words)
    return features 

In [ ]:
# Now we can extract the features and train the classifier 
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures=nltk.classify.apply_features(extract_features,preprocessedTrainingSet)

In [15]:
NBayesClassifier=nltk.NaiveBayesClassifier.train(trainingFeatures)

NameError: name 'trainingFeatures' is not defined

In [16]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

NameError: name 'preprocessedTestSet' is not defined

In [19]:
# get the majority vote
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")

NameError: name 'NBResultLabels' is not defined

In [18]:
"""
***Resources***

Authentication and Extracting tweets
https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/twitter-data-in-python/

Cursor method 
http://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html

YouTube Tutorial
https://www.youtube.com/watch?v=ujId4ipkBio
https://medium.com/better-programming/twitter-sentiment-analysis-15d8892c0082


Maybe for the ML stuff - has all the steps
https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed

"""

'\n***Resources***\n\nAuthentication and Extracting tweets\nhttps://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/twitter-data-in-python/\n\nCursor method \nhttp://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html\n\nYouTube Tutorial\nhttps://www.youtube.com/watch?v=ujId4ipkBio\nhttps://medium.com/better-programming/twitter-sentiment-analysis-15d8892c0082\n\n\nMaybe for the ML stuff - has all the steps\nhttps://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed\n\n'